In [ ]:
#This script goes through the Github repos in the csv and gets the READMEs from Github.
#The output is a csv of the README text.

import csv
import requests
import time

In [ ]:
def makereadmeurl(url):
    prefix = 'https://raw.githubusercontent.com/'
    suffix = '/master/README.md'
    spliturl = url.split('/')
    if len(spliturl) == 6:
        repo = spliturl[4] + '/' + spliturl[5]
        apiquery = prefix + repo + suffix
    else:
        apiquery = 'no repo'
    return apiquery

In [ ]:
githubapilist = []
with open('ZenodoGithubOutput.csv', 'r') as f:
    mycsv = csv.reader(f)
    for row in mycsv:
        githubapilist.append(row[10])
        #print row[10]
f.closed

In [ ]:
readmeurls = []
for row in githubapilist:
    readmeurl = makereadmeurl(row)
    readmeurls.append(readmeurl)

In [ ]:
readmetext = []
for url in readmeurls:
    try:
        results = requests.get(url)
        #print results
        readmetext.append(results.text)
    except:
        if url == 'no repo':
            #print url
            readmetext.append(url)
        else:
            #print 'investigate' + url
            readmetext.append('investigate' + url)
    time.sleep(1)

In [ ]:
#find the average length of the README files
listoflengths = []
totallength = 0
for text in readmetext:
    listoflengths.append(len(text))
for length in listoflengths:
    totallength += length
avglength = totallength/len(listoflengths)
print(avglength)

In [ ]:
readmestooutput = []
for text in readmetext:
    removednewlines = text.replace('\n', ' ').replace('\r', '').encode("utf-8")
    readmetooutput = removednewlines.decode('unicode_escape').encode('ascii','ignore')
    readmestooutput.append(readmetooutput)

In [ ]:
#remove strings of duplicate special characters
charactersremoved = []
for text in readmestooutput:
    workingtext = text
    while '==' in workingtext:
        workingtext = workingtext.replace('==', '=')
    while '--' in workingtext:
        workingtext = workingtext.replace('--', '-')
    while '**' in workingtext:
        workingtext = workingtext.replace('**', '*')
    while '##' in workingtext:
        workingtext = workingtext.replace('##', '#')
    workingtext = ' '.join(workingtext.split())
    charactersremoved.append(workingtext)

In [ ]:
with open('GithubREADMEs8190.csv', 'wb') as f:
    for i in charactersremoved:
        try:
            f.write('\"' + str(i[:8190]).replace('"', "'").replace('\\', ' ') + '\",' + str(len(i)) + '\r\n')
        except:
           f.write('\"' + i.encode('UTF-8').replace('"', "'").replace('\\', ' ') + '\" \r\n')